In [11]:
8*5*4

160

# Calculos para evaluacion de CETES

Descripción técnica de los certificados de la Tesorería de la Federación

https://www.banxico.org.mx/mercados/d/%7B0DE0044F-662D-09D2-C8B3-4F1A8E43655F%7D.pdf

**La función calcular_valores_faltantes es utilizada para calcular valores faltantes relacionados con bonos CETES. 
Los valores requeridos para el cálculo son:**

- **Valor Nominal (VN):** El valor nominal del bono, generalmente igual a 10 en el caso de CETES.
- **Número de Días (t):** El período en días hasta el vencimiento del bono.
- **Tasa de Descuento (b):** La tasa de descuento, que es una tasa anualizada utilizada para calcular el precio del bono.
- **Precio (P):** El precio del bono.
- **Tasa de Rendimiento (r):** La tasa de rendimiento anualizada del bono.
- **Descuento (D):** El valor de descuento del bono.

en donde:
![Imagen](imagenes\Precio_CETES.png)



## Para trabajar mas eficiente se crea la clase CETES que hereda las propiedades de la clase Bono.

La clase Bonos permite almacenar la informacion del bono en un diccionario llamado "_infoBono", el cual tiene como claves los mismos nombres
del vector Invex, y sus respectivos valores.

Las formulas para encontrar los calculos requeridos son ajustados a la norma

In [2]:
from INVEX_calc_Bonos import Bono

class CETES(Bono):
    
    def calcPrecioLimpio(self,r = None,t = None,VN = None):
        """
        Calcula el Precio Limpio de un bono CETES.
        Args:
            r (float, optional): Tasa de rendimiento. 
            t (int, optional): El período en días hasta el vencimiento del bono
            VN (float, optional): Valor nominal del bono. 
        Salida:
            float: El Precio Limpio calculado.
        """
        dias_año = self.days_year
        if not t: t = (self._infoBono['FechaVcto'] - self._infoBono['TimId'] ).days
        if not VN: VN = self._infoBono['ValorNominal'] 
        if not r: r = self._infoBono['TasaDeRendimiento']/100

        P = VN / (1 + r * t / dias_año)
    
        #Se guarada los valores con los que se hizo el calculo
        self._ValCalBono['PrecioLimpio'] = {'PrecioLimpio':P,'TasaDeRendimiento':r,
                                              'PlazoEmision':t,'ValorNominal':VN}
        return P
    
    def calcPrecioLimpio_TasaDescuento(self,b,t = None,VN = None):
        """
        Calcula el Precio Limpio de un bono CETES usando la tasa de descuento.
        Args:
            d (float, optional): Tasa de descuento. 
            t (int, optional): El período en días hasta el vencimiento del bono
            VN (float, optional): Valor nominal del bono. 
        Salida:
            float: El Precio Limpio calculado.
        """
        dias_año = self.days_year
        if not t: t = (self._infoBono['FechaVcto'] - self._infoBono['TimId'] ).days
        if not VN: VN = self._infoBono['ValorNominal']

        P = VN / (1 - b * t / dias_año)
            
    def calcRendimiento(self,P = None,t = None,VN = None):
        """
        Calcula la Tasa de Rendimiento de un bono CETES.
        Args:
            P (float, optional): Precio Limpio del bono. 
            t (int, optional): El período en días hasta el vencimiento del bono
            VN (float, optional): Valor nominal del bono.
        Salida:
            float: La Tasa de Rendimiento calculada.
        """
        dias_año = self.days_year
        if not t: t = (self._infoBono['FechaVcto'] - self._infoBono['TimId'] ).days
        if not VN: VN = self._infoBono['ValorNominal'] 
        if not P: P =  self._infoBono['PrecioLimpio'] 
        
        r =round(((VN/P-1)*dias_año/t)*100,6)
        
        #Se guarada los valores con los que se hizo el calculo
        self._ValCalBono['PrecioLimpio'] = {'PrecioLimpio':P,'TasaDeRendimiento':r,
                                              'PlazoEmision':t,'ValorNominal':VN}
        return r
    
    
    def calcTasaDescuento(self,r = None,t = None):
        """
        Calcula la Tasa de Descuento de un bono CETES.
        Args:
            r (float, optional): Tasa de rendimiento. 
            t (int, optional): El período en días hasta el vencimiento del bono 
            VN (float, optional): Valor nominal del bono.
        Salida:
            float: La Tasa de Descuento calculada.
        """
        dias_año = self.days_year
        if not t: t = (self._infoBono['FechaVcto'] - self._infoBono['TimId']).days
        if not r: r = self._infoBono['TasaDeRendimiento']/100
        
        b = r/(1+r*t/dias_año)
        
                
        #Se guarada los valores con los que se hizo el calculo        
        self._ValCalBono['TasaDescuento'] = {'TasaDeRendimiento':r,'PlazoEmision':t,
                                              'TasaDescuento':b}        
        return b
    
    def calcDescuento(self,P=None,VN = None):
        """
        Calcula el Descuento de un bono CETES.
        Args:
            P (float, optional): Precio Limpio del bono. 
            VN (float, optional): Valor nominal del bono. 
        Salida:
            float: El Descuento calculado.
        """
        if not VN: VN = self._infoBono['ValorNominal'] 
        if not P: P =  self._infoBono['PrecioLimpio'] 
        
        D = VN-P
        
        #Se guarada los valores con los que se hizo el calculo            
        self._ValCalBono['Descuento'] = {'PrecioLimpio':P,'Descuento':D,'ValorNominal':VN}              
        return D
    
    
    def calcRendimientoEquivalente(self,Pc,r = None,t = None,):
        """
        A partir del rendimiento de un CETE es posible obtener el rendimiento 
        implícito (también conocido como Rendimiento en Curva o Rendimiento Equivalente) 
        del mismo en un diferente plazo a vencimiento de acuerdo a la siguiente fórmula.
        
        entradas:
            r (float) = Tasa de rendimiento original del CETE en decimal.
            t (int)= Período en días hasta el vencimiento, con el que se calcula el rendimiento
            Pc (int) = Plazo en días que se desea cotizar en Curva
        salida:
            rc (float)= Rendimiento en curva o Rendimiento Equivalente en decimal.
        """
        dias_año = self.days_year
        if not r: r = self._infoBono['TasaDeRendimiento']/100
        if not t: t = (self._infoBono['FechaVcto'] - self._infoBono['TimId']).days
        
        rc = ( (1+r*t/dias_año)**(Pc/t) - 1 ) * dias_año/Pc
        #Se guarada los valores con los que se hizo el calculo            
        self._ValCalBono['RendimientoEquivalente'] = {'RendimientoEquivalente':rc,
                                                      'TasaDeRendimiento':r,'PlazoEmision':t}     
        return rc

## Pruebas

### ANEXO 2
EJEMPLO PRÁCTICO

El 31 de agosto de 2000 un inversionista compra CETES con las siguientes características:

Valor Nominal: 10.00 pesos

Fecha de Colocación: 31 de agosto de 2000

Fecha de Vencimiento: 28 de septiembre de 2000

Días por vencer del título: 28 días

Supongamos que dicho inversionista adquiere los títulos a un rendimiento anual de
15.50%. Para calcular el precio al cual tendrá que liquidar la operación, el inversionista
tiene dos opciones: a) calcular el valor presente del principal a través de la tasa de
rendimiento y b) calcular el precio a partir de la “tasa de descuento” que proporcione
este rendimiento.

In [3]:
from BI_CETES import CETES
# ANEXO 2 EJEMPLO PRÁCTICO
BonoEnEvaluacion = CETES()
BonoEnEvaluacion.modInfoBono({'ValorNominal': "10",
                              'TasaDeRendimiento':"15.50",
                              'FechaEmision':"31/08/2000", 
                              'TimId':"31/08/2000",
                              'FechaVcto':20000928,
                              'PlazoEmision':"28",
                              'PrecioLimpio':9.8808805})


print("Informacion del Bono: ")
for key in BonoEnEvaluacion.verInfoBono().keys(): 
    print("   ",key,":" ,BonoEnEvaluacion.verInfoBono()[key])
print()


P = BonoEnEvaluacion.calcPrecioLimpio()
print(f"Precio Limpio Calculado: {P:.6f},  valor esperado: {BonoEnEvaluacion.verInfoBono()['PrecioLimpio'] }")

b = BonoEnEvaluacion.calcTasaDescuento()
print(f"Tasa de descuento calculada: {b:.6f} , valor esperado: {15.32}%")

P = BonoEnEvaluacion.calcPrecioLimpio_TasaDescuento(b)
print(f"Precio Limpio usando Tasa de descuento: {P:.6f},valor esperado: {BonoEnEvaluacion.verInfoBono()['PrecioLimpio'] }")

Informacion del Bono: 
    ValorNominal : 10
    TasaDeRendimiento : 15.5
    FechaEmision : 2000-08-31
    TimId : 2000-08-31
    FechaVcto : 2000-09-28
    PlazoEmision : 28
    PrecioLimpio : 9.8808805

Precio Limpio Calculado: 9.880880,  valor esperado: 9.8808805
Tasa de descuento calculada: 0.153154 , valor esperado: 15.32%
Precio Limpio usando Tasa de descuento: 9.880880,valor esperado: 9.8808805


In [4]:
from BI_CETES import CETES

# Ejemplo general
# valor_VN, Precio, Tasa_descuento, Num_dias,Tasa_rendimiento , Descuento 
# [10,9.8575 , 0.057  , 90 ,  0.057823991884352  , 0.1425]

BonoEnEvaluacion = CETES({'ValorNominal': "10",
                              'TasaDeRendimiento':" 5.7823991884352",
                              'PrecioLimpio':9.8575 })
print("Informacion del bono: ",BonoEnEvaluacion.verInfoBono())


#Datos conocidos:
Tasa_descuento = 0.057
Num_dias_al_vencimiento = 90

# Se puede calcular el precio usando el Num_dias_al_vencimiento ya que no se tienen las fechas,
#  no es necesario introducir el rendimiento ya que esta en la informacion del bono:
P = BonoEnEvaluacion.calcPrecioLimpio(None,Num_dias_al_vencimiento)
print(f"Precio Limpio Calculado: {P:.6f},  valor esperado: {BonoEnEvaluacion.verInfoBono()['PrecioLimpio'] }")

# O el rendimiento conociendo el precio limpio:
r = BonoEnEvaluacion.calcRendimiento(None,Num_dias_al_vencimiento)
print(f"Rendimiento Calculado: {r:.6f},  valor esperado: {BonoEnEvaluacion.verInfoBono()['TasaDeRendimiento'] }")

# Tambien se puede calcularla tasa de descuento aunque esta no pertenece a los valores tipicos del vector INVEX:
b = BonoEnEvaluacion.calcTasaDescuento(None,Num_dias_al_vencimiento)
print(f"rendimiento Calculado: {b:.6f} , valor esperado: {Tasa_descuento}")


Informacion del bono:  {'ValorNominal': 10, 'TasaDeRendimiento': 5.7823991884352, 'PrecioLimpio': 9.8575}
Precio Limpio Calculado: 9.857500,  valor esperado: 9.8575
Rendimiento Calculado: 5.782000,  valor esperado: 5.7823991884352
rendimiento Calculado: 0.057000 , valor esperado: 0.057


# Probando con informacion de CETES desde vector

In [5]:
# Funcion para cargar informacion de CETES desde vector
import pandas as pd

def cargarInfoBono(num_serie,archivo = "info_Bonos\\20230831_t-1_Vector_BI.xlsx"):
    df_vectorInvex = pd.read_excel(archivo)
    bono_analizar = df_vectorInvex[df_vectorInvex['Serie'] == num_serie].reset_index(drop=True)
    info_bono_analizar = {}
    for key in bono_analizar.columns:
        val = bono_analizar.loc[0,key]
        info_bono_analizar[key] = val 
    return info_bono_analizar

#Se debe indicar un num_serie
InfoBono = cargarInfoBono(240208)

In [6]:
# La clase CETES transforma la informacion del bono para ser analizada
from BI_CETES import CETES

Serie = 240111
infoBono = cargarInfoBono(Serie)
BonoEnEvaluacion = CETES(infoBono)

print(f"Informacion del Bono Serie: {Serie}")
for key in ['FechaEmision','FechaVcto', 'PlazoEmision','ValorNominal','TasaDeRendimiento','PrecioLimpio']:
    print("   ",key,":" ,BonoEnEvaluacion.verInfoBono()[key])
print("\nResutlados de calculo:")
print("   Precio Limpio Calculado: ",BonoEnEvaluacion.calcPrecioLimpio())
print("   Rendimiento Calculado: ",BonoEnEvaluacion.calcRendimiento())  

Informacion del Bono Serie: 240111
    FechaEmision : 2023-01-12
    FechaVcto : 2024-01-11
    PlazoEmision : 364
    ValorNominal : 10
    TasaDeRendimiento : 11.48
    PrecioLimpio : 9.593134

Resutlados de calculo:
   Precio Limpio Calculado:  9.59313387
   Rendimiento Calculado:  11.48


## Revision de todos los bonos en vector

In [7]:
import pandas as pd
import math

def series(archivo = "info_Bonos\\20230831_t-1_Vector_BI.xlsx"):
    df_vectorInvex = pd.read_excel(archivo)
    serie = [ x for x in ['Serie','SERIE'] if x in df_vectorInvex.columns][0]
    return list(df_vectorInvex[serie])

tol = 1e-6
Error = False
for num_serie in series():
    infoBono = cargarInfoBono(num_serie)
    BonoEnEvaluacion = CETES(infoBono)
    PrecioLimpioCalculado = BonoEnEvaluacion.calcPrecioLimpio()
    PrecioLimpioVector = BonoEnEvaluacion.verInfoBono()['PrecioLimpio'] 
    RendimientoCalculado = BonoEnEvaluacion.calcRendimiento()
    RendimientoVector =  BonoEnEvaluacion.verInfoBono()['TasaDeRendimiento']
    if (not math.isclose(PrecioLimpioCalculado, PrecioLimpioVector, rel_tol=tol, abs_tol=tol) or not math.isclose(RendimientoCalculado, RendimientoVector, rel_tol=tol, abs_tol=tol)):
        Error = True
        print(f"Serie: {num_serie}, PrecioLimpioCalculado =  {PrecioLimpioCalculado}, PrecioLimpioVector = {PrecioLimpioVector},RendimientoCalculado =  {RendimientoCalculado}, RendimientoVector = {RendimientoVector} ")
        
if not Error:
    print(f"Todos los calculos coinciden con una exactitud de {tol} digitos significativos")

Todos los calculos coinciden con una exactitud de 1e-06 digitos significativos


## Calculos de Regresos

In [8]:
help(BonoEnEvaluacion.CalcRegresos)

Help on method CalcRegresos in module BI_CETES:

CalcRegresos(TasaReporto, plazoReporto, PrecioSucio=None, r=None, t=None, VN=None) method of BI_CETES.CETES instance
    Calculos para regresos, calcula el valor futuro del precio del CETE con la tasa de reporto 
    y por el lapso del prestamo a reporto.
    Entradas : 
        - TasaReporto (float): Tasa de reporto en %
        - plazoReporto (int): Cantidad de dias del prestamo a reporto
        - PrecioSucio (float, optional): Se toma de la informacion de Bono
                                (En caso de CETES es el mismo precio limpio)
        - Rendimiento (float, optional): rendimiento anual esperado por el inversionista en %.
        - t (int, optional): El período en días hasta el vencimiento del bono
        - VN (float, optional): Valor nominal del bono.



In [9]:
from datetime import  timedelta,datetime
from BI_CETES import CETES
import pandas as pd

def cargarInfoBono(num_serie,archivo = "info_Bonos\\20230831_t-1_Vector_BI.xlsx "):
    df_vectorInvex = pd.read_excel(archivo)
    bono_analizar = df_vectorInvex[df_vectorInvex['Serie'] == num_serie].reset_index(drop=True)
    info_bono_analizar = {}
    for key in bono_analizar.columns:
        val = bono_analizar.loc[0,key]
        info_bono_analizar[key] = val 
    return info_bono_analizar

#Cargando informacion del Bono
Serie = 240111
infoBono = cargarInfoBono(Serie)
BonoEnEvaluacion = CETES(infoBono)

#Datos para calculo de regreso
TasaReporto = 11.25
plazoReporto = 10

PrecioLimpioCalculado = BonoEnEvaluacion.calcPrecioLimpio()
PrecioLimpioVector = BonoEnEvaluacion.verInfoBono()['PrecioLimpio'] 
RendimientoCalculado = BonoEnEvaluacion.calcRendimiento()
RendimientoVector =  BonoEnEvaluacion.verInfoBono()['TasaDeRendimiento']

print(f"PrecioLimpioCalculado =  {PrecioLimpioCalculado}, PrecioLimpioVector = {PrecioLimpioVector},RendimientoCalculado =  {RendimientoCalculado}, RendimientoVector = {RendimientoVector} ")
print()
PrecioRegreso = BonoEnEvaluacion.CalcRegresos(TasaReporto, plazoReporto)
print(f"Precio de regreso: {PrecioRegreso}" )
RendimientoRegreso = BonoEnEvaluacion.calcRendimiento(PrecioRegreso)
print(f"Tasa regreso: {RendimientoRegreso}" )

PrecioLimpioCalculado =  9.59313387, PrecioLimpioVector = 9.593134,RendimientoCalculado =  11.48, RendimientoVector = 11.48 

Precio de regreso: 9.623154736227466
Tasa regreso: 10.6


## DETERMINACIÓN DEL RENDIMIENTO EN CURVA O TASA EQUIVALENTE
### Tomado de anexo 3 LS_BPAG182

A partir del rendimiento de un CETE es posible obtener el rendimiento implícito (también
conocido como Rendimiento en Curva o Rendimiento Equivalente) del mismo en un
diferente plazo a vencimiento de acuerdo a la siguiente fórmula:

![Imagen](imagenes\CETES_TASA_EQUIVALENTE.png)

donde:

rc = Rendimiento en curva

r = Tasa de rendimiento original del CETE

p = Plazo original en días del CETE

pC = Plazo en días que se desea cotizar en Curva


In [10]:
from BI_CETES import CETES

# EJEMPLO PRÁCTICO
BonoEnEvaluacion = CETES()
Pc = 91
r = 0.1392
t = 28 
rc = BonoEnEvaluacion.calcRendimientoEquivalente(Pc,r,t)
print("Rendimiento en curva = ", round(rc*100,4))

Rendimiento en curva =  14.0903
